In [1]:
from utils import data_process,confusion_matrix,visualize_
from math import log
from random import randint,choices

In [2]:
path = "..//data//train.csv"
X_train,y_train,X_test,y_test,Dict = data_process(path)
feature_cat = [3,6,7,8,9,10,11,13,14,15,16,17,18,21,23,29,30,31,32,33,34,35,36,37,38,39,40]
feature_num = []
for i in range(42): #ID and is_claim is excluded
    if i not in feature_cat:
        feature_num.append(i)

In [3]:
seq = []
for i in range(len(X_train)):
    seq.append(i)

In [4]:
sample_list = []
for i in range(1,len(feature_cat)+len(feature_num)):
    sample_list.append(i)

In [5]:
def log2(p):
    return log(p)/log(2)
    
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * log2(p) + (1 - p) * log2(1-p))

def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r

In [6]:
def slices(data,indices):
    d = []
    for ind in indices:
        d.append(data[ind])
    return d

In [7]:
def draw_bootstrap(X_train, y_train):
    bootstrap_indices = choices(seq,k=len(X_train))
    oob_indices = list(set(seq)-set(bootstrap_indices))
    X_bootstrap = slices(X_train,bootstrap_indices)
    y_bootstrap = slices(y_train,bootstrap_indices)
    X_oob = slices(X_train,oob_indices)
    y_oob = slices(y_train,oob_indices)
    return X_bootstrap, y_bootstrap, X_oob, y_oob

In [8]:
def oob_score(tree, X_test, y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

In [9]:
def sampling(size):
    length = len(sample_list)
    for i in range(length):
        ind1 = randint(0,length-1)
        ind2 = randint(0,length-1)
        temp = sample_list[ind1]
        sample_list[ind1] = sample_list[ind2]
        sample_list[ind2] = temp
    ind_start = randint(0,length-7)
    return sample_list[ind_start:ind_start+6]

In [10]:
def data_col(data,feature_ind,Dict):
    col_data = []
    if feature_ind in feature_cat: # +1 because the key value in Dict didn't exclude ID
        max_counter = len(Dict[feature_ind+1])
        ls = []
        for i in range(max_counter):
            ls.append(i)
    else:
        ls = set()
        for d in data:
            ls.add(d[feature_ind])
    
    for d in data:
        col_data.append(d[feature_ind])
    
    return col_data,ls

In [11]:
def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = sampling(max_features)

    best_info_gain = -999
    node = None
    for feature_ind in feature_ls:
        col_data,unique_col_data = data_col(X_bootstrap,feature_ind,Dict)
        for split_point in unique_col_data:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}

            # split children for continuous variables
            if feature_ind in feature_num:
                for i, value in enumerate(col_data):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
            
            # split children for categorical variables
            else:
                for i, value in enumerate(col_data):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])

            split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                node = {'information_gain': split_info_gain,
                        'left_child': left_child,
                        'right_child': right_child,
                        'split_point': split_point,
                        'feature_ind': feature_ind}

    return node

In [12]:
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred


def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_features, min_samples_split, max_depth, depth + 1)
    
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

In [13]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

def random_forest(X_train, y_train, n_estimators, max_depth, min_samples_split, max_features):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split,max_features)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate: {:.2f}".format(sum(oob_ls)/len(oob_ls)))# mean
    return tree_ls

In [14]:
def predict_tree(tree, X_test):
    feature_ind = tree['feature_ind']

    if X_test[feature_ind] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

In [15]:
def predict_rf(tree_ls, X_test):
    pred_ls = []
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return pred_ls

In [16]:
n_estimators = 10
max_features = int(42**0.5)
max_depth = 20
min_samples_split = 2

model = random_forest(X_train, y_train, n_estimators, max_depth, min_samples_split, max_features)

OOB estimate: 0.12


In [17]:
preds = predict_rf(model, X_test)


TypeError: 'bool' object is not iterable

In [22]:
preds==y_test

False